In [1]:
import csv
import numpy as np
import pandas as pd

## ETF Data Handling

In [2]:
import datetime
import time

In [3]:
"""
# TODO: 
Get those etf name which exists before 12/31/2015
It should be:
    
etf_name_list = ['XLE', 'AMLP', 'VDE', 'AMJ', 'EMLP', 'XOP', 'MLPI', 'IXC', 'OIH', 'MLPA', 'IYE', 'MLPX', 'FENY', 'AMZA', 'ERX', 'ATMP', 'IEO', 'TPYP', 'AMU', 'TAN', 'GUSH', 'GREK', 'XES', 'RYE', 'FXN', 'ICLN', 'IEZ', 'PZD', 'PBW', 'DIG', 'FCG', 'MLPC', 'QCLN', 'GEX', 'FAN', 'KOL', 'ENFR', 'PXI', 'ZMLP', 'PBD', 'FILL', 'MLPY', 'DRIP', 'PXE', 'YMLP', 'CRAK', 'IMLP', 'GASL', 'MLPO', 'FRAK', 'PSCE', 'GRID', 'NLR', 'ERY', 'YMLI', 'PXJ', 'YLCO', 'PUW', 'DUG', 'ENY', 'MLPG', 'MLPB', 'AMUB', 'GASX', 'CHIE', 'DDG']
len(etf_name_list) == 66
    
try it !!
"""

etf_name_list = []
df = pd.read_csv("Energy Equity ETF List (80).csv")
date = { "name": df["Symbol"], "date": df["Inception"] }
date = pd.DataFrame(date)
#print(date.head())

#轉換時間格式
for a in range(0,len(date)):
    t = time.strptime(date["date"][a], "%d/%m/%Y")
    date["date"][a] = time.strftime("%Y-%m-%d", t)
#print(date.head()) 

#篩選日期
mask = date["date"] <= "2015-12-31"
etf_name_list = list(date[mask]["name"])
    
print(etf_name_list)
len(etf_name_list)

['XLE', 'AMLP', 'VDE', 'AMJ', 'EMLP', 'XOP', 'MLPI', 'IXC', 'OIH', 'MLPA', 'IYE', 'MLPX', 'FENY', 'AMZA', 'ERX', 'ATMP', 'IEO', 'TPYP', 'AMU', 'TAN', 'GUSH', 'GREK', 'XES', 'RYE', 'FXN', 'ICLN', 'IEZ', 'PZD', 'PBW', 'DIG', 'FCG', 'MLPC', 'QCLN', 'GEX', 'FAN', 'KOL', 'ENFR', 'PXI', 'ZMLP', 'PBD', 'FILL', 'MLPY', 'DRIP', 'PXE', 'YMLP', 'CRAK', 'IMLP', 'GASL', 'MLPO', 'FRAK', 'PSCE', 'GRID', 'NLR', 'ERY', 'YMLI', 'PXJ', 'YLCO', 'PUW', 'DUG', 'ENY', 'MLPG', 'MLPB', 'AMUB', 'GASX', 'CHIE', 'DDG']


66

## Web Crawler

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import requests


start_day = 1451577600       # Dec 31, 2015
#end_day = int(time.time())   # Now()
end_day = 1552665600  #Mar 15, 2015
per_day = 86400
month_trans_dict = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

my_options = Options()
my_options.add_argument("--headless")
driver = webdriver.Chrome('./chromedriver.exe', chrome_options=my_options)

def get_statistics(etf_name):
    statistic = []
    time_list = []
    period2 = end_day
    #period1 = start_day
    period1 = end_day - per_day * 90
    while True:
        break_flag = False
        if period1 <= start_day:
            period1 = start_day
            break_flag = True
        url = 'https://finance.yahoo.com/quote/%s/history?period1=%d&period2=%d&interval=1d&filter=history&frequency=1d'%(etf_name, period1, period2)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, features='lxml')
        tr_list = soup.find_all('tr', {"class": "BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)"})
        for tr in tr_list:
            td_list = tr.find_all('td', {'class': 'Py(10px) Pstart(10px)'})
            if len(td_list) < 1:
                continue
            td_time = tr.find('td', {'class': 'Py(10px) Ta(start) Pend(10px)'})
            td_time = re.split(r'[,\s]', td_time.get_text())
            time_format = '%04d-%02d-%02d'%(int(td_time[3]), month_trans_dict[td_time[0]], int(td_time[1]))
            time_list.append(time_format)
            statistic.append(td_list[4].get_text())
        if break_flag:
            break
        period2 = period1 - per_day
        period1 = period2 - per_day * 90
        #print('-----')
    #print(len(statistic), len(time_list))
    statistic.reverse()
    time_list.reverse()
    df = pd.DataFrame({'Date': time_list, etf_name: statistic})
    return df

def get_FENY():
    '''
    #my_options = Options()
    #my_options.add_argument("--headless")
    #driver = webdriver.Chrome('./chromedriver.exe', chrome_options=my_options)
    #driver = webdriver.Chrome('chromedriver.exe')
    driver.get("https://ETF.com/FENY/#overview")
    driver.find_element_by_link_text("Fund Home Page").click()
    driver.find_element_by_id("priceHistory").click()
    driver.find_element_by_link_text("Download to CSV file").click()
    time.sleep(6)
    #driver.quit()
    print("Successfully download FENY \n")
    '''
    #dateframe
    df = pd.read_csv("FENY_Price_History_20190315.csv")
    df = df[550:1356]
    df = df.iloc[:,[0,4]]
    #df = df.iloc[::-1]
    df = df.rename(columns={'Close':'FENY'})
    for a in range(550,1356):
        t = time.strptime(df["Date"][a], "%m/%d/%Y")
        df["Date"][a] = time.strftime("%Y-%m-%d", t)

    FENY = pd.DataFrame(df)
    return FENY

def get_TPYP(): 
    r = requests.get("http://etp.tortoiseadvisors.com/tpyp/premium-discount/")
    if r.status_code == requests.codes.ok:
        soup = BeautifulSoup(r.text, 'html.parser')
    tags = soup.find_all('table')
    df = pd.read_html(str(tags[0]))
    dff = pd.DataFrame(df[0])
    for a in range(0,904):
        t = time.strptime(dff["Date"][a], "%m/%d/%Y")
        dff["Date"][a] = time.strftime("%Y-%m-%d", t)

    a = dff[0:781] #date filter
    TPYP = a.iloc[:,[0,1]] #keep date & NAV
    TPYP = TPYP.iloc[::-1]
    TPYP = TPYP.rename(columns={'Share Price':'TPYP'})
    return TPYP


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


In [5]:
def get_all_statistics(etf_name_list):
    statistics = []
    dataFrame_time = []
    dataFrame_column = []
    #etf_name_list = ['XLE', 'TPYP', 'FENY']
    df_all = []
    empty = True
    for etf_name in etf_name_list:
        print('#### start handle %s ####'%(etf_name))
        if etf_name == 'FENY':
            df = get_FENY()
        elif etf_name == 'TPYP':
            df = get_TPYP()
        else:
            if empty:
                df_all = get_statistics(etf_name)
                empty = False
                continue
            else:
                df = get_statistics(etf_name)
        df_all = pd.merge(df_all, df, on='Date', how='outer')
    return df_all

## Main

In [6]:
df_all = get_all_statistics(etf_name_list)
print(df_all)

#### start handle XLE ####
#### start handle AMLP ####
#### start handle VDE ####
#### start handle AMJ ####
#### start handle EMLP ####
#### start handle XOP ####
#### start handle MLPI ####
#### start handle IXC ####
#### start handle OIH ####
#### start handle MLPA ####
#### start handle IYE ####
#### start handle MLPX ####
#### start handle FENY ####


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### start handle AMZA ####
#### start handle ERX ####
#### start handle ATMP ####
#### start handle IEO ####
#### start handle TPYP ####


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### start handle AMU ####
#### start handle TAN ####
#### start handle GUSH ####
#### start handle GREK ####
#### start handle XES ####
#### start handle RYE ####
#### start handle FXN ####
#### start handle ICLN ####
#### start handle IEZ ####
#### start handle PZD ####
#### start handle PBW ####
#### start handle DIG ####
#### start handle FCG ####
#### start handle MLPC ####
#### start handle QCLN ####
#### start handle GEX ####
#### start handle FAN ####
#### start handle KOL ####
#### start handle ENFR ####
#### start handle PXI ####
#### start handle ZMLP ####
#### start handle PBD ####
#### start handle FILL ####
#### start handle MLPY ####
#### start handle DRIP ####
#### start handle PXE ####
#### start handle YMLP ####
#### start handle CRAK ####
#### start handle IMLP ####
#### start handle GASL ####
#### start handle MLPO ####
#### start handle FRAK ####
#### start handle PSCE ####
#### start handle GRID ####
#### start handle NLR ####
#### start handle ERY ####
#### start